In [1]:
# Import necessary libraries
from ultralytics import YOLO
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from PIL import Image as PILImage
import io
from IPython.display import Image, display
from PIL import Image as PILImage
import numpy as np
import tensorflow as tf
import io
import os
import pandas as pd
import torch
import json
from torch import float32, tensor
import torchmetrics
import cv2


2023-12-13 11:09:50.117986: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 11:09:50.118011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 11:09:50.118604: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
import shutil

# Source directory containing images
src_dir = './TRAIN'

# Destination directory to copy images
dst_dir = './TEST'

# Check if destination directory exists, create it if it doesn't
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# Loop through all files in source directory
for file_name in os.listdir(src_dir):
    # Check if file ends with .jpg
    if len(os.listdir(dst_dir)) == 50:
        break
    if file_name.endswith('.jpg'):
        # Construct full file paths
        src_path = os.path.join(src_dir, file_name)
        dst_path = os.path.join(dst_dir, file_name)
        # Copy file to destination directory
        shutil.copy(src_path, dst_path)




In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the YOLO model
yolo_model = YOLO('./best.pt').to(device)
# Load the ResNet50 model
model2 = load_model('./model_resnet50.h5')

# Class mapping
class_mapping = {'Apple Scab Leaf': 0, 'Apple leaf': 1, 'Apple rust leaf': 2, 'Bell_pepper leaf': 3, 'Bell_pepper leaf spot': 4, 'Blueberry leaf': 5, 'Cherry leaf': 6, 'Corn Gray leaf spot': 7, 'Corn leaf blight': 8, 'Corn rust leaf': 9, 'Peach leaf': 10, 'Potato leaf': 11, 'Potato leaf early blight': 12, 'Potato leaf late blight': 13, 'Raspberry leaf': 14, 'Soyabean leaf': 15, 'Squash Powdery mildew leaf': 16, 'Strawberry leaf': 17, 'Tomato Early blight leaf': 18, 'Tomato Septoria leaf spot': 19, 'Tomato leaf': 20, 'Tomato leaf bacterial spot': 21, 'Tomato leaf late blight': 22, 'Tomato leaf mosaic virus': 23, 'Tomato leaf yellow virus': 24, 'Tomato mold leaf': 25, 'Tomato two spotted spider mites leaf': 26, 'grape leaf': 27, 'grape leaf black rot': 28}

# Directory containing images to predict
directory = './TEST'

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df = pd.read_csv('set.csv')

# Define a list of dictionaries
data = []
# Create empty lists to store results
resized_cropped_images = []
predicted_classes = []

# Define the target size for ResNet50
target_size = (256, 256)
preds=[]
# Process YOLO results
for filename in os.listdir(directory):
    filtered_df = df[df['filename'] == filename]
    results = yolo_model.predict(os.path.join(directory,filename))
    for x in results:
        for r in x:
            # Get the image and bounding box
            im_array = r.plot()  # plot a BGR numpy array of predictions
            im = PILImage.fromarray(im_array[..., ::-1])  # Convert BGR to RGB PIL image
            bbox = r.boxes.data[0, :4].cpu().numpy()  # Get the bounding box

            # Crop the detected object from the image
            cropped_im = im.crop((bbox[0], bbox[1], bbox[2], bbox[3]))

            # Resize the cropped image to the target size
            resized_cropped_im = cropped_im.resize(target_size)
            # plt.imshow(resized_cropped_im)

            # Save the resized and cropped image to a BytesIO object
            img_io = io.BytesIO()
            resized_cropped_im.save(img_io, 'PNG')

            # Append the resized and cropped image to the list
            resized_cropped_images.append(resized_cropped_im)

            # Predict the class of the cropped object using the loaded ResNet50 model
            img_array = np.array(resized_cropped_im)
            img_array = tf.image.resize(img_array, (256, 256))
            img_array = np.expand_dims(img_array, axis=0)
            predictions = model2.predict(img_array)
            predicted_class_index = np.argmax(predictions, axis=1)
            predicted_class_label = list(class_mapping.keys())[predicted_class_index[0]]
            predicted_classes.append(predicted_class_label)
            # Convert bounding box to tensor and move to GPU
            bbox_tensor = torch.tensor([[bbox[0], bbox[1], bbox[2], bbox[3]]]).to(device)
            
            # Convert labels to tensor and move to GPU
            label_tensor = torch.tensor(predicted_class_index).to(device)

            # Convert scores to tensor and move to GPU
            score_tensor = torch.tensor([predictions.max()]).to(device)
            minind=0
            q=0
            err=1e9
            for(index, row) in filtered_df.iterrows():
                # Access fields using the column names
                temerr=(bbox[0]-row['xmin'])**2+(bbox[1]-row['ymin'])**2+(bbox[2]-row['xmax'])**2+(bbox[3]-row['ymax'])**2
                if(temerr<err):
                    err=temerr
                    minind=q
                q+=1
            print(len(filtered_df), " ", minind)
            if(len(filtered_df)>0):
                data.append({'boxes': torch.tensor([[(filtered_df.iloc[minind]['xmin'].copy()), (filtered_df.iloc[minind]['ymin'].copy()), (filtered_df.iloc[minind]['xmax'].copy()), (filtered_df.iloc[minind]['ymax'].copy())]]).to(device), 'labels': torch.tensor([(class_mapping[filtered_df.iloc[minind]['class']])]).to(device)})
                # filtered_df.drop(filtered_df.index[minind], inplace=True)
                preds.append(dict(boxes=bbox_tensor, labels=label_tensor, scores=score_tensor))
    filtered_df=[]
    
    


image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/TEST/image022.jpg: 480x640 4 leafs, 63.2ms
Speed: 2.0ms preprocess, 63.2ms inference, 234.6ms postprocess per image at shape (1, 3, 480, 640)
1/1 [==============================] - 2s 2s/step
5   2
1/1 [==============================] - 0s 14ms/step
5   1
1/1 [==============================] - 0s 15ms/step
5   3
1/1 [==============================] - 0s 14ms/step
5   4

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/TEST/green-pepper-plant-diseases-bacterial-spot-on-pepper-plant-green-pepper-plant-diseases-photos.jpg: 480x640 5 leafs, 6.4ms
Speed: 1.6ms preprocess, 6.4ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)
1/1 [==============================] - 0s 15ms/step
5   1
1/1 [==============================] - 0s 15ms/step
5   3
1/1 [==============================] - 0s 14ms/step
5   0
1/1 [==============================] - 0s 16ms/step
5   2
1/1 [==============================] - 

3   2

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/TEST/cedar%20apple%20rust%20lower%20leaf%20haw%20S%202%20ne%20OH%20RA%209-2-16.jpg: 480x640 1 leaf, 7.0ms
Speed: 1.7ms preprocess, 7.0ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)
1/1 [==============================] - 0s 14ms/step
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/TEST/Jessore-potato-late-blight-8-2h2fdkl.jpg: 640x480 4 leafs, 7.0ms
Speed: 2.6ms preprocess, 7.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)
1/1 [==============================] - 0s 14ms/step
3   0
1/1 [==============================] - 0s 15ms/step
3   1
1/1 [==============================] - 0s 16ms/step
3   0
1/1 [==============================] - 0s 19ms/step
3   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/TEST/ebleafb.jpg: 448x640 1 leaf, 6.2ms
Speed: 1.9ms preprocess, 6.2ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)

1/1 [==============================] - 0s 15ms/step
5   4
1/1 [==============================] - 0s 15ms/step
5   4
1/1 [==============================] - 0s 19ms/step
5   4

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/TEST/20130515+008.jpg: 480x640 8 leafs, 7.8ms
Speed: 2.2ms preprocess, 7.8ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
1/1 [==============================] - 0s 14ms/step
11   3
1/1 [==============================] - 0s 17ms/step
11   1
1/1 [==============================] - 0s 15ms/step
11   7
1/1 [==============================] - 0s 14ms/step
11   0
1/1 [==============================] - 0s 15ms/step
11   2
1/1 [==============================] - 0s 15ms/step
11   4
1/1 [==============================] - 0s 15ms/step
11   0
1/1 [==============================] - 0s 15ms/step
11   3

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/TEST/0peach.jpg: 416x640 2 leafs, 63.6ms
Speed: 2.0ms preprocess, 63.6ms infer

In [6]:
# for img_data, predicted_class in zip(resized_cropped_images, predicted_classes):
#     display(img_data, predicted_class, class_mapping[predicted_class])


In [7]:
metrics = torchmetrics.detection.MeanAveragePrecision().to(device)
for i in range(len(preds)):
    print(preds[i])
    print(data[i])
metrics.update(preds, data)
print(metrics.compute())

{'boxes': tensor([[100.8355,  35.8091, 173.5686,  82.3700]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.7311], device='cuda:0')}
{'boxes': tensor([[ 94,  33, 174,  82]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}
{'boxes': tensor([[2.2778e+02, 2.2078e-01, 3.4100e+02, 6.8144e+01]], device='cuda:0'), 'labels': tensor([6], device='cuda:0'), 'scores': tensor([0.9428], device='cuda:0')}
{'boxes': tensor([[232,   2, 334,  63]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}
{'boxes': tensor([[197.5413,  49.8215, 252.8159, 131.8659]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9991], device='cuda:0')}
{'boxes': tensor([[193,  57, 260, 143]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}
{'boxes': tensor([[136.1880,  62.6873, 213.2086, 217.6336]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.4742], device='cuda:0')}
{'boxes': tensor([[139,  78, 205,

{'boxes': tensor([[  8.6313,  83.5316, 315.9281, 225.4954]], device='cuda:0'), 'labels': tensor([21], device='cuda:0'), 'scores': tensor([0.9859], device='cuda:0')}
{'boxes': tensor([[ 22,  90, 325, 223]], device='cuda:0'), 'labels': tensor([12], device='cuda:0')}
{'boxes': tensor([[289.3352, 213.4701, 378.3626, 301.7247]], device='cuda:0'), 'labels': tensor([28], device='cuda:0'), 'scores': tensor([0.7370], device='cuda:0')}
{'boxes': tensor([[287, 212, 379, 296]], device='cuda:0'), 'labels': tensor([12], device='cuda:0')}
{'boxes': tensor([[261.1261,  80.3796, 379.0000, 146.9505]], device='cuda:0'), 'labels': tensor([5], device='cuda:0'), 'scores': tensor([0.8242], device='cuda:0')}
{'boxes': tensor([[261,  82, 373, 152]], device='cuda:0'), 'labels': tensor([12], device='cuda:0')}
{'boxes': tensor([[219.9592, 129.8989, 377.3402, 263.0590]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.6101], device='cuda:0')}
{'boxes': tensor([[287, 212, 379, 296]], 

{'boxes': tensor([[420.2488, 443.3892, 682.5640, 740.0229]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9950], device='cuda:0')}
{'boxes': tensor([[388, 445, 678, 740]], device='cuda:0'), 'labels': tensor([27], device='cuda:0')}
{'boxes': tensor([[722.9599, 384.4512, 996.8743, 685.7546]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.7884], device='cuda:0')}
{'boxes': tensor([[718, 155, 879, 411]], device='cuda:0'), 'labels': tensor([27], device='cuda:0')}
{'boxes': tensor([[5.9593e+02, 2.2908e-01, 8.2125e+02, 1.8357e+02]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9999], device='cuda:0')}
{'boxes': tensor([[718, 155, 879, 411]], device='cuda:0'), 'labels': tensor([27], device='cuda:0')}
{'boxes': tensor([[740.0131,   0.0000, 922.1865, 221.0314]], device='cuda:0'), 'labels': tensor([6], device='cuda:0'), 'scores': tensor([0.4298], device='cuda:0')}
{'boxes': tensor([[718, 155, 879,

{'boxes': tensor([[ 289.7924,  699.7946,  751.7421, 1271.2563]], device='cuda:0'), 'labels': tensor([5], device='cuda:0'), 'scores': tensor([1.0000], device='cuda:0')}
{'boxes': tensor([[ 292,  705,  765, 1286]], device='cuda:0'), 'labels': tensor([5], device='cuda:0')}
{'boxes': tensor([[ 92.5731, 274.6984, 381.1708, 624.1040]], device='cuda:0'), 'labels': tensor([6], device='cuda:0'), 'scores': tensor([0.4928], device='cuda:0')}
{'boxes': tensor([[ 150,  624,  553, 1071]], device='cuda:0'), 'labels': tensor([5], device='cuda:0')}
{'boxes': tensor([[ 190.6139,  625.6694,  537.3990, 1047.8676]], device='cuda:0'), 'labels': tensor([5], device='cuda:0'), 'scores': tensor([1.0000], device='cuda:0')}
{'boxes': tensor([[ 150,  624,  553, 1071]], device='cuda:0'), 'labels': tensor([5], device='cuda:0')}
{'boxes': tensor([[  2.1368, 501.6848, 406.6104, 803.4506]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.7986], device='cuda:0')}
{'boxes': tensor([[ 150,  

{'boxes': tensor([[  1.4268, 192.6890, 481.1201, 552.3919]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.7400], device='cuda:0')}
{'boxes': tensor([[  1, 201, 493, 554]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}
{'boxes': tensor([[501.4277,  91.7126, 790.3455, 412.3945]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([1.0000], device='cuda:0')}
{'boxes': tensor([[505,  81, 801, 459]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}
{'boxes': tensor([[ 415.2290,  809.7595, 1209.3079, 1079.4940]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9891], device='cuda:0')}
{'boxes': tensor([[ 710,  501, 1170,  827]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}
{'boxes': tensor([[   0.0000,  585.5546,  321.9730, 1081.0000]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.7470], device='cuda:0')}
{'boxes': tensor([[ 216,  39

{'boxes': tensor([[ 62.2950, 183.2696, 187.9368, 297.8766]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.9994], device='cuda:0')}
{'boxes': tensor([[ 70, 187, 196, 299]], device='cuda:0'), 'labels': tensor([8], device='cuda:0')}
{'boxes': tensor([[238.8278, 127.3986, 340.3435, 199.4420]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.9819], device='cuda:0')}
{'boxes': tensor([[117, 183, 348, 299]], device='cuda:0'), 'labels': tensor([8], device='cuda:0')}
{'boxes': tensor([[141.5871, 179.8192, 309.4443, 275.3711]], device='cuda:0'), 'labels': tensor([7], device='cuda:0'), 'scores': tensor([0.3933], device='cuda:0')}
{'boxes': tensor([[117, 183, 348, 299]], device='cuda:0'), 'labels': tensor([8], device='cuda:0')}
{'boxes': tensor([[ 62.2300, 116.1589, 268.0089, 198.3311]], device='cuda:0'), 'labels': tensor([10], device='cuda:0'), 'scores': tensor([0.4013], device='cuda:0')}
{'boxes': tensor([[ 35, 115, 269, 194]], dev

{'boxes': tensor([[548.7242,   4.6026, 664.9478, 238.1573]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.9899], device='cuda:0')}
{'boxes': tensor([[398, 143, 645, 330]], device='cuda:0'), 'labels': tensor([3], device='cuda:0')}
{'boxes': tensor([[  0.0000, 239.5029, 103.1067, 489.0625]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.4452], device='cuda:0')}
{'boxes': tensor([[ 45, 396, 230, 525]], device='cuda:0'), 'labels': tensor([3], device='cuda:0')}
{'boxes': tensor([[  1.6560, 168.7656, 465.9230, 556.3681]], device='cuda:0'), 'labels': tensor([19], device='cuda:0'), 'scores': tensor([0.9835], device='cuda:0')}
{'boxes': tensor([[  3, 176, 462, 548]], device='cuda:0'), 'labels': tensor([19], device='cuda:0')}
{'boxes': tensor([[138.4027,  38.2761, 750.7568, 562.3962]], device='cuda:0'), 'labels': tensor([16], device='cuda:0'), 'scores': tensor([0.7156], device='cuda:0')}
{'boxes': tensor([[391, 373, 757, 528]], d